![alt text](
https://upgini.com/lib_tHloTHmnvYomfhRQ/1lth2xdahxnz5tr4.svg?w=206)

# Quick Start guide: Search new relevant external features for  product sales forecast  
_________________

Following this guide, you'll learn how to **search new relevant features with Upgini low-code library**. We will enrich a dataset with new features and significantly improve model accuracy. All in 4 simple steps.  
The goal is to predict future sales of different goods in stores based on a 5-year history of sales. The evaluation metric is SMAPE.  
Time needed: *15 minutes.*  

Download this notebook: [GitHub Link](https://github.com/upgini/upgini/blob/main/notebooks/kaggle_example.ipynb)
_________________

First, let's install latest version of Upgini library. Also, we'll need CatBoost for the last part of this guide.

In [ ]:
%pip install -Uq upgini catboost

## 1️⃣ Prepare input data

For this guide we'll use the train dataset from [Store Item Demand Forecasting Challenge](https://www.kaggle.com/c/demand-forecasting-kernels-only). You can download it from [here](https://www.kaggle.com/c/demand-forecasting-kernels-only/data?select=train.csv).  
To speed up the search we'll take a subsample.  
⚠️ All columns in the input dataset with dates/datetime should be converted to pandas datetime object for correct datetime representation

In [ ]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

# Convert date column to datetime pandas object
df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


This dataset contains 5 years of records from 2013 to 2017. Let's split it into the train (2013–2016) and the evaluation (2017) parts.

In [ ]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

Let's also separate features from targets in *a scikit-learn style* for future use.

In [ ]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

## 2️⃣ Search new relevant features with FeaturesEnricher

Next, we will use **`FeaturesEnricher`** on the train dataset to find new features relevant for this target prediction.  
* To do this, we need to specify the column(s) containing [**search key(s)**](https://github.com/upgini/upgini#-search-key-types-we-support-more-is-coming), in this case it's `date` and provide the target to predict.  
* Also, we can specify any number of additional out-of-time validation datasets to evaluate robustness of the new features.  
* This search task will be auto-detected as a regression. And as we have time series prediction (daily sales as a target variable), we have to pass [**time series specific cross-validation split**](https://github.com/upgini/upgini#-time-series-prediction-support) **`CVType.time_series`**. Now search algorithm know that we are working with the time series prediction task, not just simple regression and will use [time series CV](https://scikit-learn.org/stable/modules/cross_validation.html#time-series-split) for new features search.  

Search step will take around *2.5 minutes*

In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys = {
      "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)]
)

Detected task type: ModelTaskType.REGRESSION


,Column name,Status,Description
0,target,All valid,All values in this column are good to go
1,date,All valid,All values in this column are good to go


Running search request with search_id=61edb3e6-a159-4037-816a-30e6e7539d08
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done                          


We found 25 useful feature(s) for you by search keys: ['date']


,feature_name,shap_value,coverage %,type
0,item,0.488219,100.0,CHARACTER
1,store,0.171972,100.0,CHARACTER
2,f_weather_pca_0_94efd18d,0.103756,100.0,NUMERIC
3,f_week_cos1_d3d56d7f,0.028162,100.0,NUMERIC
4,f_year_cos1_cd165f8c,0.020851,100.0,NUMERIC
5,f_week_sin1_a71d22f6,0.020216,100.0,NUMERIC
6,f_payment_fraud_score_3cae9c42,0.016986,100.0,NUMERIC
7,f_c2c_fraud_score_5028232e,0.015011,100.0,NUMERIC
8,f_silver_d4264cf9,0.010140,100.0,NUMERIC
9,f_cpi_pca_2_3c36cd6c,0.006687,100.0,NUMERIC


We've got **10+ new relevant features** from [different sources such as weather data, calendar data, financial data](https://github.com/upgini/upgini#-connected-data-sources-and-coverage), which expected to improve accuracy of the model. Ranked by [SHAP values](https://en.wikipedia.org/wiki/Shapley_value).

Initial features from search dataset will be checked for relevancy as well, so you don't need an extra feature selection step.

## 3️⃣ Calculate model metrics and uplift from new relevant features
You can use any model estimator with scikit-learn compatible interface. Let's take CatBoost regressor.  
For evaluation metric there are two options:
* Predefined evaluation function alias from [*Upgini library*](https://github.com/upgini/upgini#-accuracy-and-uplift-metrics-calculations), like **`RMSLE`** for Root Mean Squared Logarithmic Error

* Define custom evaluation function using [scikit-learn make_scorer](https://scikit-learn.org/0.15/modules/model_evaluation.html#defining-your-scoring-strategy-from-score-functions), for example [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error)

Model evaluation metric both for train and validation datasets will be calculated with the same cross-validation strategy as for **`FeaturesEnricher.fit()`**-  in this example [time series CV](https://github.com/upgini/upgini#-time-series-prediction-support). 

In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

# Calculate metrics before and after enrichment with new relevant features
enricher.calculate_metrics(
    train_features, train_target, 
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

,match_rate,baseline mean_absolute_percentage_error,enriched mean_absolute_percentage_error,uplift
,,,,
train,100.0,0.255844,0.171971,0.083873
eval 1,100.0,0.243877,0.139098,0.104778


We've got a strong metric uplift both on the cross-validation (*train*) and on the out-of-time validation dataset (*eval1*) **after enrichment**.

## 4️⃣ Enrich datasets with the new features and retrain model

Now we can enrich our datasets with the features found and use them in our own ML pipelines. Lets' enrich both the train and the test datasets.  
Enrichment step for two datasets will take *2.5 minutes*

In [ ]:
enriched_train_features = enricher.transform(train_features, keep_input = True)
enriched_test_features = enricher.transform(test_features, keep_input = True)
enriched_train_features.head()

90.39637% of the rows are fully duplicated


,Column name,Status,Description
0,date,All valid,All values in this column are good to go


Running search request with search_id=0927144a-e0da-4f5e-bcf4-8d80c4af5f17
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done                          

Executing transform step
Done                          
90.36176% of the rows are fully duplicated


,Column name,Status,Description
0,date,All valid,All values in this column are good to go


Running search request with search_id=df715aa2-5325-46f3-8163-0e23e1c4ee86
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done                          

Executing transform step
Done                          


,date,store,item,f_weather_pca_0_94efd18d,f_week_cos1_d3d56d7f,f_year_cos1_cd165f8c,f_week_sin1_a71d22f6,f_payment_fraud_score_3cae9c42,f_c2c_fraud_score_5028232e,f_silver_d4264cf9,...,f_weather_umap_30_98fa4f7d,f_stoxx_4ddb0c38,f_nasdaq_d309709a,f_transaction_fraud_union_score_f9bc093c,f_italy_game_cnt_9cfcfe65,f_silver_7d_to_7d_1y_shift_ccbd2abf,f_dow_jones_7d_to_7d_1y_shift_9628c89b,f_finance_umap_0_ad818bcb,f_weather_umap_11_c213a9d7,f_finance_pca_1_d9e67c86
0,2013-01-01,7,5,28.661328,0.62349,0.98522,0.781831,0.232837,0.369604,30.173,...,3.547175,278.779999,3019.51001,0.177833,0,1.08019,1.065812,10.026849,7.594507,0.98871
1,2013-01-01,4,9,28.661328,0.62349,0.98522,0.781831,0.232837,0.369604,30.173,...,3.547175,278.779999,3019.51001,0.177833,0,1.08019,1.065812,10.026849,7.594507,0.98871
2,2013-01-01,1,33,28.661328,0.62349,0.98522,0.781831,0.232837,0.369604,30.173,...,3.547175,278.779999,3019.51001,0.177833,0,1.08019,1.065812,10.026849,7.594507,0.98871
3,2013-01-01,3,41,28.661328,0.62349,0.98522,0.781831,0.232837,0.369604,30.173,...,3.547175,278.779999,3019.51001,0.177833,0,1.08019,1.065812,10.026849,7.594507,0.98871
4,2013-01-01,5,24,28.661328,0.62349,0.98522,0.781831,0.232837,0.369604,30.173,...,3.547175,278.779999,3019.51001,0.177833,0,1.08019,1.065812,10.026849,7.594507,0.98871


We've got new features and ready to retrain the model.  
**BEFORE** enrichment with the new features:

In [ ]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[37.65141857448004]

**AFTER** enrichment:

In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE")

[14.858872412911566]

______________________________
Thanks for reading! If you found this useful or interesting, please share with a friend.
______________________________
## 🔗 Useful links
* Upgini Library [Documentation](https://github.com/upgini/upgini#readme)
* More notebooks and [Guides](https://github.com/upgini/upgini#-quick-start-and-guides)
* Kaggle public [Notebooks](https://www.kaggle.com/romaupgini/code)


<sup>😔 Found mistype or a bug in code snippet? Our bad! <a href="https://github.com/upgini/upgini/issues/new?assignees=&title=readme%2Fbug">
Please report it here.</a></sup>